In [1]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime
import csv


In [2]:
driver = GraphDatabase.driver('bolt://localhost/:7687', auth=('neo4j', 'rcai'))

In [3]:
def get_labels(dbname, dbdriver)->[]:
    query = "CALL db.labels()"
    with dbdriver.session(database=dbname) as session:
        results = session.run(query)
        df = pd.DataFrame(results.values())
    labels = df[0].values.tolist()
    labels.sort()
    return labels

def get_label_counts(dbname, dbdriver)->dict:
    result = {"Label": [], "Count": []}
    labels = get_labels(dbname, dbdriver)
    with dbdriver.session(database=dbname) as session:
        for label in labels:
            query = f"MATCH (:`{label}`) RETURN count(*) as count"
            results = session.run(query)
            count = pd.DataFrame(results.values()).iloc[0][0]
            result['Label'].append(label)
            result['Count'].append(count)
    return pd.DataFrame(data=result).sort_values("Label")

def compare_node_counts(dbnames:[], dbdriver)->pd.DataFrame:
    df_combo = pd.DataFrame()
    for dbname in dbnames:
        df = get_label_counts(dbname, dbdriver)
        df.set_index('Label', inplace=True)
        df = df.rename(columns={'Count':dbname})
        df_combo = df_combo.join(df, how='outer')
    df_combo.reset_index(inplace=True)
    return df_combo

In [4]:
def get_relationship_counts(dbname, dbdriver):
    result = {"RelationshipType": [], "Count": []}
    with dbdriver.session(database=dbname) as session:
        results = session.run("CALL db.relationshipTypes()")
        rels = pd.DataFrame(results.values())[0]
        for rel in rels.values:
            query = f"MATCH ()-[:`{rel}`]->() RETURN count(*) as count"
            count = pd.DataFrame(session.run(query).values())[0][0]
            result["RelationshipType"].append(rel)
            result["Count"].append(count)
    rels_df = pd.DataFrame(data=result).sort_values("Count", ascending=False)
    return rels_df

def compare_relationship_counts(dbnames:[], dbdriver):
    df_combo = pd.DataFrame()
    for dbname in dbnames:
        rels_df = get_relationship_counts(dbname, dbdriver)
        rels_df.set_index('RelationshipType', inplace=True)
        rels_df = rels_df.rename(columns={'Count': dbname})
        df_combo = df_combo.join(rels_df, how='outer')
    df_combo.reset_index(inplace=True)
    return df_combo

In [7]:
def compare_databases(dbnames:[], driver, outfile):
    df1 = compare_node_counts(dbnames, driver)
    df2 = compare_relationship_counts(dbnames, driver)
    print('writing', outfile)
    with pd.ExcelWriter(outfile, engine='xlsxwriter') as writer:
        df1.to_excel(writer, sheet_name='nodes')
        df2.to_excel(writer, sheet_name='edges')
    

In [8]:
date_tag = datetime.today().strftime('%m%d%y')
outfile = f'/Users/rcai/data/notebook/general/ecocyc_db_compare_{date_tag}.xlsx'
dbnames = [
    'ecocyc', 'ecotest',
    'ecocyc-gds-nocollapse', 'ecotest-no-collapse',
    'ecocyc-reg-collapse', 'ecotest-reg-collapse',
    'ecocyc-25.5-gds', 'ecotest-gds'
]
compare_databases(dbnames, driver, outfile)

writing /Users/rcai/data/notebook/general/ecocyc_db_compare_062222.xlsx
